In [21]:
del li
#del logs_df

In [4]:
# Importamos bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime
import re
from sklearn.ensemble import IsolationForest

# Establecemos las columnas
colz=["IP","timestamp","status code","bytes sent","user agent"]

li=[]
c=0

# Abrimos el archivo
with open("../logs/access.log") as f:
    while(True):
        if(c==100000):
            break
        if(c%10000==0):
            print(c) # Esto nose, yo lo borraria
        try:
            x=f.readline()
            if x==None:
                break
            log=x.split(" ")
            if(len(log)<12):
                continue
            #print([log[0],log[3].strip('['),log[5].strip('"'),log[7].strip('"'),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
            li.append([log[0],log[3].strip('['),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
        except Exception as e:
            pass
            #print(e)
            #print(log)
            #print(int(log[8]),int(log[9]))
            #print("skipped an entry")
        c+=1

# basic feature engineering
logs_df=pd.DataFrame(li,columns=colz)
logs_df["timestamp"]=pd.to_datetime(logs_df["timestamp"],format='%d/%b/%Y:%H:%M:%S')

#logs_df["url_length"]=logs_df["URL"].apply(len)
#logs_df["url_depth"]=logs_df["URL"].apply(lambda x : x.count('/'))
#logs_df['num_encoded_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'%[0-9A-Fa-f]{2}', x)))
#logs_df['num_special_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'[|,;]', x)))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [5]:
from ua_parser import user_agent_parser

def parse_ua(ua):
    parsed = user_agent_parser.Parse(ua)
    so = parsed["os"]["family"]
    marca = parsed["device"]["brand"]
    modelo = parsed["device"]["model"]
    return pd.Series([so, marca, modelo])

logs_df[["so", "marca", "modelo"]] = logs_df["user agent"].apply(parse_ua)

In [12]:
c= 0
for x in logs_df['user agent']:
    print(x)
    c+=1
    if c == 40:
        break

Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)
Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36
Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16.0)Gecko/16.0 Firefox/16.0
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.ht

In [18]:
logs_df

,IP,timestamp,status code,bytes sent,user agent,so,marca,modelo
0,54.36.149.41,2019-01-22 03:56:14,200,30577,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,Other,Spider,Desktop
1,31.56.96.51,2019-01-22 03:56:16,200,5667,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,Android,Huawei,ALE-L21
2,31.56.96.51,2019-01-22 03:56:16,200,5379,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,Android,Huawei,ALE-L21
3,40.77.167.129,2019-01-22 03:56:17,200,1696,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,Other,Spider,Desktop
4,91.99.72.15,2019-01-22 03:56:17,200,41483,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,Windows,None,None
...,...,...,...,...,...,...,...,...
99992,95.81.72.148,2019-01-22 08:01:41,200,5450,Mozilla/5.0 (Windows NT 6.1; rv:60.0) Gecko/20...,Windows,None,None
99993,95.81.72.148,2019-01-22 08:01:41,200,3568,Mozilla/5.0 (Windows NT 6.1; rv:60.0) Gecko/20...,Windows,None,None
99994,204.18.201.252,2019-01-22 08:01:41,200,19241,Mozilla/5.0 (Linux; Android 8.1.0; SM-G610F) A...,Android,Samsung,SM-G610F
99995,95.81.72.148,2019-01-22 08:01:41,200,2581,Mozilla/5.0 (Windows NT 6.1; rv:60.0) Gecko/20...,Windows,None,None


In [19]:
# Nos quedamos solamente con las variables que nos interesan
dispositivos = logs_df[["so", "marca", "modelo"]]

# Eliminamos duplicados
dispositivos = dispositivos.drop_duplicates().reset_index(drop=True)

# Añadimos el ID
dispositivos.insert(0, "id_dispositivo", range(1, len(dispositivos) + 1))

In [20]:
# Vemos la tabla resultante
dispositivos

,id_dispositivo,so,marca,modelo
0,1,Other,Spider,Desktop
1,2,Android,Huawei,ALE-L21
2,3,Windows,None,None
3,4,Android,HTC,Desire 728 dual sim
4,5,Linux,None,None
...,...,...,...,...
546,547,Android,Generic_Android,L454
547,548,Android,Generic_Android,TECNO-W5
548,549,Android,HTC,Desire 816G dual sim
549,550,Android,Lenovo,A7600-HV


In [ ]:
# Actualizamos la dimension Dispositivos en el DW
from Modulos.update_dimensions_table import actualizarTablaDimension
dispositivos = actualizarTablaDimension(engine, "dispositivos", dispositivos, pk="id_dispositivo")